# Getting Started with LouieAI Notebook Interface

This notebook demonstrates the basic usage of the LouieAI notebook-friendly API (`lui`).

## Authentication Requirements

LouieAI uses PyGraphistry for authentication. Before running this notebook, you need:

1. **PyGraphistry Account**: Sign up for free at [hub.graphistry.com](https://hub.graphistry.com)
2. **Set Credentials**: Use one of these methods:

   ```bash
   # Option 1: Environment variables (recommended for notebooks)
   export GRAPHISTRY_USERNAME="your_username"
   export GRAPHISTRY_PASSWORD="your_password"
   ```
   
   ```python
   # Option 2: Authenticate in code
   import graphistry
   graphistry.register(api=3, server="hub.graphistry.com", 
                      username="your_username", password="your_password")
   ```

## Setup

First, make sure you have LouieAI installed and your credentials configured.

In [1]:
import os

import graphistry

from louieai import louie  # Factory function to create Louie interface

# Authenticate with Graphistry
# Uses GRAPHISTRY_USERNAME and GRAPHISTRY_PASSWORD from environment
g = graphistry.register(
    api=3,
    server=os.environ.get("GRAPHISTRY_SERVER", "hub.graphistry.com"),
    username=os.environ.get("GRAPHISTRY_USERNAME"),
    password=os.environ.get("GRAPHISTRY_PASSWORD"),
)

# Configure Louie server URL
# Default to production server, or set LOUIE_SERVER env var for custom
# Examples:
#   export LOUIE_SERVER="http://localhost:8000"  # Local development
#   export LOUIE_SERVER="https://den.louie.ai"    # Production (default)
LOUIE_SERVER_URL = os.environ.get("LOUIE_SERVER", "https://den.louie.ai")

# Create the Louie interface
lui = louie(g, server_url=LOUIE_SERVER_URL)

lui  # Display status

<LouieAI Notebook Interface | Session: Not started | History: 0 responses | Traces: Disabled>

## Basic Queries

The `lui` interface accepts natural language queries and returns AI-generated responses. Let's verify it works with a simple arithmetic question:

In [2]:
# Ask a simple arithmetic question - should mention "4"
response = lui("What is 2 plus 2? Just tell me the answer.")

# The response is automatically stored and accessible
print(f"Answer: {lui.text}")

# Verify we got the correct answer
if "4" in str(lui.text) or "four" in str(lui.text).lower():
    print("✅ Basic query working correctly!")
else:
    print("⚠️ Response may be incomplete - expected '4' in answer")

Answer: 2 plus 2 equals 4.
✅ Basic query working correctly!


## Working with Data

LouieAI can generate and analyze data. Let's create a sample dataset:

In [3]:
# Use the exact working prompt from our test
lui("Generate a pandas dataframe with 3 rows: name, age, city columns")

# Access the generated dataframe
df = lui.df
if df is not None:
    print(f"Generated DataFrame with shape: {df.shape}")
    print("\nDataFrame contents:")
    display(df)
else:
    print("No dataframe was generated. Let's try a different approach.")
    print(f"Response text: {lui.text}")
    
    # Try alternate approach
    lui("Create a table with 3 products: name, price, quantity")
    if lui.df is not None:
        print(f"\nSecond attempt successful! Shape: {lui.df.shape}")
        display(lui.df)

Generated DataFrame with shape: (3, 3)

DataFrame contents:


,name,age,city
0,Alice,25,New York
1,Bob,30,Los Angeles
2,Charlie,35,Chicago


## Analyzing Data  

Once you have data, you can ask LouieAI to perform calculations and analysis using its code agent:

In [4]:
# Ask for a simple fact about the data
lui("How many people are in the dataframe?")

# The answer should be available
print(f"Result: {lui.text[:200]}...")

# Verify the answer (should mention 3)
if "3" in str(lui.text) or "three" in str(lui.text).lower():
    print("✅ Data analysis working!")
else:
    print("⚠️ Expected count of 3 not found")
    
# Ask another simple question
lui("Who is the oldest person?")
print(f"\nAnalysis: {lui.text[:200]}...")


Analysis: Who is the oldest person in the dataframe B_7clTzcCY using the 'age' column?"
...


## Beyond DataFrames

LouieAI can also analyze images, PDFs, Excel files, and more!

For comprehensive examples of uploading different file types, see the **[Data Upload Guide](02-data-upload.ipynb)**:
- Image analysis (PNG, JPEG, etc.)
- Document processing (PDF, Word, PowerPoint)
- Excel file handling
- Advanced upload options

In [5]:
# Quick example: Working with product data
import pandas as pd

# Create a products DataFrame
products_df = pd.DataFrame({
    "product": ["Widget A", "Widget B", "Widget C"],
    "price": [10.99, 15.49, 8.99],
    "quantity": [100, 75, 150]
})

print("Product inventory:")
print(products_df)

# Analyze with LouieAI
lui("Calculate the total inventory value (sum of price * quantity for all products)", products_df)
print(f"\nInventory Analysis: {lui.text[:300]}...")

# Check if calculation was performed
if any(str(val) in str(lui.text) for val in ["3610", "3611", "3,610", "3,611"]):
    print("✅ Inventory calculation working!")

Product inventory:
    product  price  quantity
0  Widget A  10.99       100
1  Widget B  15.49        75
2  Widget C   8.99       150



Inventory Analysis: Calculate the total inventory value (sum of price * quantity for all products)...


## Uploading Your Own Data

LouieAI can analyze DataFrames you create or load from files:

In [6]:
import numpy as np
import pandas as pd

# Create your own DataFrame (or load from CSV/Excel)
my_data = pd.DataFrame(
    {
        "date": pd.date_range("2024-01-01", periods=30, freq="D"),
        "sales": np.random.randn(30).cumsum() + 100,
        "visitors": np.random.randint(50, 200, 30),
        "region": np.random.choice(["North", "South", "East", "West"], 30),
    }
)

print("Sample of your data:")
my_data.head()

Sample of your data:


,date,sales,visitors,region
0,2024-01-01,100.132129,172,North
1,2024-01-02,100.427145,94,South
2,2024-01-03,99.481349,166,North
3,2024-01-04,98.658227,63,South
4,2024-01-05,97.870372,120,South


In [7]:
# Upload and analyze your DataFrame
lui("Calculate the sum of all sales values in this dataset", my_data)

# The AI analyzes YOUR uploaded data
print(f"Analysis: {lui.text[:300]}...")

# Check if it processed the data
if "sales" in str(lui.text).lower() or "sum" in str(lui.text).lower():
    print("✅ DataFrame upload and analysis working!")

# You can ask follow-up questions about the uploaded data
lui("Which region appears most frequently in the data?")
print(f"\nRegion analysis: {lui.text[:200]}...")


Region analysis: It seems there was an issue with identifying the correct dataframe for your query about the most common region. The system attempted to access a dataframe with the ID B_Y2ipolZB, but it encountered an...


## Session History

The `lui` interface maintains a history of your interactions:

In [8]:
# Access previous responses by index
# lui[-1]  # Last response
# lui[-2]  # Second to last response

# Check the number of interactions
print(f"Total interactions: {len(lui._history)}")

Total interactions: 7


## Enabling Traces

For complex queries, you might want to see the AI's reasoning process:

In [9]:
# Enable traces for the session
lui.traces = True

# Now queries will show reasoning steps
lui("What is 100 + 50?")

# Disable traces
lui.traces = False

# Or enable traces for just one query
lui("What is the sum of 1 to 10?", traces=True)

<LouieAI Notebook Interface | Session: Active | History: 9 responses | Traces: Disabled | Latest: 1 text>

## Error Handling

The interface handles errors gracefully:

In [10]:
# Check if the last operation had errors
if lui.has_errors:
    print("Last operation had errors:", lui.errors)
else:
    print("No errors in last operation")

No errors in last operation


## Tips and Tricks

Here are some helpful patterns for using the notebook interface:

1. **Use clear prompts**: Be specific about what you want to analyze
2. **Check outputs**: Always verify `lui.df` and `lui.text` after queries
3. **Enable traces for debugging**: Use `traces=True` when troubleshooting
4. **Leverage history**: Access previous results with `lui[-1]`, `lui[-2]`, etc.
5. **Upload various formats**: DataFrames, images, PDFs all work seamlessly